In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from fastai.basics import *
from fastai.gen_doc.nbdoc import *
from fastai.vision import *

In [0]:
def normy(y):
    if y.shape[0] == 1:
        y = (y//255)
    return y

norm_y = TfmPixel(normy)

_tfms = get_transforms()
tfms = [_tfms[0]+[norm_y()], _tfms[1]+[norm_y()]]

data = (SegmentationItemList.from_folder('/content/drive/My Drive/Images/X')
       .split_by_rand_pct()
       .label_from_func(lambda path: path.parent.parent/'Y'/f'{path.stem[:]}.png', classes=np.array(['Background','Traffic Light']))
       .transform(tfms, tfm_y=True)
       .databunch(bs=16, path='/content/drive/My Drive/Neuro')
       .normalize(imagenet_stats))

In [0]:
learn = unet_learner(data, models.resnet18, metrics=dice)
learn.fit_one_cycle(4,1e-2)
learn.save('mini_train')
learn.export()

In [0]:
learn.show_results(ds_type=DatasetType.Train, rows=4, figsize=(10,10))